In [ ]:
%load_ext autoreload

In [97]:
import os
import csv
import cv2
from localizer import *
from models import *
%autoreload

Load Data

In [25]:
def preprocess_image(image):
    # resize 244 x 244 x 3
    image = cv2.resize(image, (244, 244))
    return image
def scale_rssi(rssi):
    return (rssi + 100) * (1./70.)
def scale_xy():
    pass

In [26]:
lab_localizer = localizer()
lab_localizer.make_profile('test_prof', length=12)
print(lab_localizer)

Creating new profile: test_prof
----------------------------------------
Localizer
Wifi Device: wlan0
Profile: test_prof Length: 12
Routers:
    260453955621905
    260453955621904
    260453955621906
    260453955621907
    36168625175133
    13640875216817
    13640875216816
    13640875216818
    13640875216819
    48751578567808
    13640875210497
    13640875210496
----------------------------------------


In [55]:

def load_data(path, profile=None):
    wifi = []   # (len, num_cells_in_profile)
    mag = []    # (len, 2)
    img = []    # (len, 2, imgx, imgy, imgd)
    xy = []     # (len, 2)
    file_count = 0
    if profile is not None:
        for file_name in os.listdir(path):
            if os.path.isfile(path + file_name):
                print('-- loading from {}'.format(file_name))
                with open(path + file_name, 'r') as f:
                    reader = csv.reader(f)
                    for row in reader:
                        xy.append([float(row[0]), float(row[1])])
                        mag.append([float(row[2]), float(row[3])])
                        img1 = cv2.imread(row[5])
                        img1 = preprocess_image(img1)
                        #img2 = cv2.imread(row[6])
                        #img2 = preprocess_image(img2)
                        img.append(img1)
                        count = 0
                        wifi_row = []
                        for addr in profile:
                            for cell in row[7:]:
                                mac, rssi = cell.split(' ')
                                if addr == mac:
                                    wifi_row.append(scale_rssi(float(rssi)))
                                    break
                            else:
                                wifi_row.append(scale_rssi(-100.0))
                        wifi.append(wifi_row)
            file_count += 1
        print('Loaded {} files from {}'.format(file_count, path))
        return np.array(wifi), np.array(mag), np.array(img), np.array(xy)
        
wifi, mag, images, y = load_data('../datasets/', lab_localizer.profile)

print(wifi.shape)
print(mag.shape)
print(images.shape)
print(y.shape)

-- loading from 2017-07-13_15:14
-- loading from 2017-07-13_11:06
-- loading from 2017-07-13_15:15
Loaded 5 files from ../datasets/
(110, 12)
(110, 2)
(110, 244, 244, 3)
(110, 2)


Load Models

In [39]:
print(wifi[0])
print(mag[0])
#cv2.imshow('img', images[0,0])
#cv2.waitKey(0)
print(y[0])

[ 0.8         0.8         0.8         0.8         0.          0.71428571
  0.71428571  0.71428571  0.71428571  0.68571429  0.54285714  0.54285714]
[ 0.  0.]
[ 0.  0.]


Train

In [107]:
from models import *
%autoreload
model = TriInput((None, 12), (None, 2), (244, 244, 3), 2).model

/home/nvidia/git-repos/localizer/scripts/models.py:96: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (1, 1), input_shape=(244, 244,...)`
  branch3.add(Convolution2D(32, 1, 1, input_shape=self.img_input_shape))
/home/nvidia/git-repos/localizer/scripts/models.py:99: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, (2, 2))`
  branch3.add(Convolution2D(16, 2, 2))
/home/nvidia/git-repos/localizer/scripts/models.py:107: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  model.add(Merge([branch1, branch2, branch3], mode=Concatenate(axis=-1)))


ValueError: "concat" mode can only merge layers with matching output shapes except for the concat axis. Layer shapes: [(None, None, 32), (None, None, 4), (None, 944784)]

Visualize